In [28]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn import tree 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets, linear_model, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

pd.set_option('display.float_format',lambda x: '%.4f' %x)
pd.options.mode.chained_assignment = None

train_data = pd.read_csv('data/trainingset.txt',header=None)
predict_data = pd.read_csv('data/queries.txt',header=None)
#set column names for train_data
train_data.columns = ['id','age','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome','output']
predict_data.columns = ['id','age','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome','output']

#assign the datatypes to the columns
train_data.age = train_data.age.astype('int64')
train_data.job = train_data.job.astype('category')
train_data.marital = train_data.marital.astype('category')
train_data.education = train_data.education.astype('category')
train_data.default = train_data.default.map({'yes':1,'no':0})
train_data.balance = train_data.balance.astype('float64')
train_data.housing = train_data.housing.map({'yes': 1, 'no': 0})
train_data.loan = train_data.loan.map({'yes': 1, 'no': 0})
train_data.contact = train_data.contact.map({'cellular': 0, 'telephone': 1, 'unknown': 2})
train_data.day = train_data.day.astype('int64')
train_data.month = train_data.month.astype('category')
train_data.duration = train_data.duration.astype('int64')
train_data.campaign = train_data.campaign.astype('int64')
train_data.pdays = train_data.pdays.astype('int64')
train_data.previous = train_data.previous.astype('int64')
train_data.poutcome = train_data.poutcome.map({'failure': 0, 'success': 1, 'unknown': 2})
train_data.output = train_data.output.map({'TypeA': 1, 'TypeB': 0})

#assign the same datatypes to the columns of the predict_data
predict_data.age = predict_data.age.astype('int64')
predict_data.job = predict_data.job.astype('category')
predict_data.marital = predict_data.marital.astype('category')
predict_data.education = predict_data.education.astype('category')
predict_data.default = predict_data.default.map({'yes':1,'no':0})
predict_data.balance = predict_data.balance.astype('float64')
predict_data.housing = predict_data.housing.map({'yes': 1, 'no': 0})
predict_data.loan = predict_data.loan.map({'yes': 1, 'no': 0})
predict_data.contact = predict_data.contact.map({'cellular': 0, 'telephone': 1, 'unknown': 2})
predict_data.day = predict_data.day.astype('int64')
predict_data.month = predict_data.month.astype('category')
predict_data.duration = predict_data.duration.astype('int64')
predict_data.campaign = predict_data.campaign.astype('int64')
predict_data.pdays = predict_data.pdays.astype('int64')
predict_data.previous = predict_data.previous.astype('int64')
predict_data.poutcome = predict_data.poutcome.map({'failure': 0, 'success': 1, 'unknown': 2})
predict_data.output = predict_data.output.map({'TypeA': 1, 'TypeB': 0})

#convert the catagorical variables to dummy variables
train_data = pd.get_dummies(train_data, columns=['job','marital','education','default','housing','loan','contact','month','poutcome'],drop_first=True)
predict_data = pd.get_dummies(predict_data, columns=['job','marital','education','default','housing','loan','contact','month','poutcome'],drop_first=True)


In [29]:
#create a predict function that uses KNN to predict the output column in predict_data
#then write the predictions to a txt file using the layout <id>,<output> where <id> is the id and <output> is either TypeA or TypeB
def predict_knn(predict_data):
    #split the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(train_data.drop(['id','output'], axis=1), train_data['output'], test_size=0.2, random_state=2)
    
    #create a KNN model
    clf = KNeighborsClassifier(n_neighbors=4, p =2 )
    #fit the model to the training data
    clf = clf.fit(X_train, y_train)
    #predict the output column in predict_data
    y_pred = clf.predict(X_test)
    #test the accuracy of the model
    print("accuracy score: ",accuracy_score(y_test, y_pred))

    # variance score: 1 means perfect prediction
    print('Variance score: {}'.format(clf.score(X_test, y_test)))  

    #print out the error rates
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

    #predict the output column in predict_data
    predict_data['output'] = clf.predict(predict_data.drop(['id','output'], axis=1))
    #round the predict_data['output'] to nearest 1 or 0
    predict_data['output'] = predict_data['output'].map({1:'TypeA', 0:'TypeB'})
    predict_data.to_csv('testdata.txt', sep=',', header=True, index=False)
    #drop all the columns except id and output
    output_data = predict_data[['id','output']]
    #write the predictions to a txt file
    output_data.to_csv('predictions.txt', sep=',', header=False, index=False)
    return predict_data, output_data

#call the function predict_knn()
predict_data, output_data = predict_knn(predict_data)
output_data['output'].value_counts()



accuracy score:  0.852796052631579
Variance score: 0.852796052631579
Mean Absolute Error: 0.14720394736842105
Mean Squared Error: 0.14720394736842105
Root Mean Squared Error: 0.383671666100614


TypeA    2476
TypeB     227
Name: output, dtype: int64

In [30]:
#create a function to cross validate the model
def cross_validate(train_data):
    #split the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(train_data.drop(['id','output'], axis=1), train_data['output'], test_size=0.2, random_state=2)
    sScalar = StandardScaler()
    X_train = sScalar.fit_transform(X_train)
    X_test = sScalar.transform(X_test)
    #create a list of the KNN model with different values of K
    K = range(1,30)
    #create an empty list to store the scores of the KNN model with different values of K
    scores = []
    #loop through each value of K
    for k in K:
        #create a KNN model
        clf = KNeighborsClassifier(n_neighbors=k)
        #fit the model to the training data
        clf = clf.fit(X_train, y_train)
        #append the scores of the model to scores
        scores.append(clf.score(X_test, y_test))
        print("KNN with k = ",k,": ",scores[k-1])
    #plot the scores of the KNN model with different values of K
    plt.figure()
    plt.plot(K, scores, 'bx-')
cross_validate(train_data)

KNN with k =  1 :  0.8447779605263158
KNN with k =  2 :  0.7962582236842105
KNN with k =  3 :  0.8813733552631579
KNN with k =  4 :  0.8694490131578947
KNN with k =  5 :  0.8916529605263158
KNN with k =  6 :  0.887952302631579
KNN with k =  7 :  0.8943256578947368
KNN with k =  8 :  0.8955592105263158
KNN with k =  9 :  0.8955592105263158
KNN with k =  10 :  0.895764802631579
KNN with k =  11 :  0.8965871710526315
KNN with k =  12 :  0.8961759868421053
KNN with k =  13 :  0.8959703947368421
KNN with k =  14 :  0.8955592105263158
KNN with k =  15 :  0.8974095394736842
KNN with k =  16 :  0.8961759868421053
KNN with k =  17 :  0.8965871710526315
KNN with k =  18 :  0.8967927631578947
KNN with k =  19 :  0.8955592105263158
KNN with k =  20 :  0.8976151315789473
KNN with k =  21 :  0.8959703947368421
KNN with k =  22 :  0.8978207236842105
KNN with k =  23 :  0.8961759868421053
KNN with k =  24 :  0.897203947368421
KNN with k =  25 :  0.8961759868421053
KNN with k =  26 :  0.896792763157894

In [ ]:

#create a function that uses linear regression to predict the output column in predict_data
#then write the predictions to a txt file using the layout <id>,<output> where <id> is the id and <output> is either TypeA or TypeB
def predict(predict_data):
    #split the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(train_data.drop(['id','output'], axis=1), train_data['output'], test_size=0.35, random_state=4)
    #create a linear regression model
    regr = linear_model.LinearRegression()
    #fit the model to the training data
    regr.fit(X_train, y_train)
    #predict the output column in predict_data
    y_pred = regr.predict(X_test)
    print(y_pred)
    #test the accuracy of the model
    print("accuracy score: ",accuracy_score(y_test, y_pred.round()))

    # variance score: 1 means perfect prediction
    print('Variance score: {}'.format(regr.score(X_test, y_test)))  

    #print out the error rates
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

    #write the predictions to a txt file using the layout <id>,<output> where <id> is the id and <output> is either TypeA or TypeB
    #predict the output column in predict_data
    predict_data['output'] = regr.predict(predict_data.drop(['id','output'], axis=1)).round()
    #round the predict_data['output'] to nearest 1 or 0
    predict_data['output'] = predict_data['output'].map({1:'TypeA', 0:'TypeB'})
    predict_data.to_csv('testdata.txt', sep=',', header=True, index=False)
    #drop all the columns except id and output
    output_data = predict_data[['id','output']]
    #write the predictions to a txt file
    output_data.to_csv('predictions.txt', sep=',', header=False, index=False)

    return predict_data, output_data
predict_data, output_data = predict(predict_data)
output_data['output'].value_counts()

[0.84391735 0.8595865  0.44375889 ... 0.93863874 0.95455155 0.88418637]
accuracy score:  0.8966165413533834
Variance score: 0.18199210785877262
Mean Absolute Error: 0.171966554856117
Mean Squared Error: 0.08473701104471518
Root Mean Squared Error: 0.2910962229997414


TypeA    2625
TypeB      78
Name: output, dtype: int64

In [ ]:
#create a predict function that uses decision tree to predict the output column in predict_data
#then write the predictions to a txt file using the layout <id>,<output> where <id> is the id and <output> is either TypeA or TypeB
def predict_tree(predict_data):
    #split the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(train_data.drop(['id','output'], axis=1), train_data['output'], test_size=0.35, random_state=4)
    #create a decision tree model
    clf = tree.DecisionTreeClassifier()
    #fit the model to the training data
    clf = clf.fit(X_train, y_train)
    #predict the output column in predict_data
    y_pred = clf.predict(X_test)
    #test the accuracy of the model
    print("accuracy score: ",accuracy_score(y_test, y_pred))

    # variance score: 1 means perfect prediction
    print('Variance score: {}'.format(clf.score(X_test, y_test)))  

    #print out the error rates
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

    #write the predictions to a txt file using the layout <id>,<output> where <id> is the id and <output> is either TypeA or TypeB
    #predict the output column in predict_data
    predict_data['output'] = clf.predict(predict_data.drop(['id','output'], axis=1))
    #round the predict_data['output'] to nearest 1 or 0
    predict_data['output'] = predict_data['output'].map({1:'TypeA', 0:'TypeB'})
    predict_data.to_csv('testdata.txt', sep=',', header=True, index=False)
    #drop all the columns except id and output
    output_data = predict_data[['id','output']]
    #write the predictions to a txt file
    output_data.to_csv('predictions.txt', sep=',', header=False, index=False)
    return predict_data, output_data

#call the function predict()
predict_data, output_data = predict_tree(predict_data)
output_data['output'].value_counts()

accuracy score:  0.8348214285714286
Variance score: 0.8348214285714286
Mean Absolute Error: 0.16517857142857142
Mean Squared Error: 0.16517857142857142
Root Mean Squared Error: 0.40642166702646576


TypeA    2372
TypeB     331
Name: output, dtype: int64

In [ ]:
#create a predict function that uses Naive Bayes to predict the output column in predict_data
#then write the predictions to a txt file using the layout <id>,<output> where <id> is the id and <output> is either TypeA or TypeB
def predict_nb(predict_data):
    #split the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(train_data.drop(['id','output'], axis=1), train_data['output'], test_size=0.35, random_state=4)
    #create a naive bayes model
    clf = GaussianNB()
    
    #fit the model to the training data
    clf = clf.fit(X_train, y_train)
    #predict the output column in predict_data
    y_pred = clf.predict(X_test)
    #test the accuracy of the model
    print("accuracy score: ",accuracy_score(y_test, y_pred))

    # variance score: 1 means perfect prediction
    print('Variance score: {}'.format(clf.score(X_test, y_test)))  

    #print out the error rates
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

    #write the predictions to a txt file using the layout <id>,<output> where <id> is the id and <output> is either TypeA or TypeB
    #predict the output column in predict_data
    predict_data['output'] = clf.predict(predict_data.drop(['id','output'], axis=1))
    #round the predict_data['output'] to nearest 1 or 0
    predict_data['output'] = predict_data['output'].map({1:'TypeA', 0:'TypeB'})
    predict_data.to_csv('testdata.txt', sep=',', header=True, index=False)
    #drop all the columns except id and output
    output_data = predict_data[['id','output']]
    #write the predictions to a txt file
    output_data.to_csv('predictions.txt', sep=',', header=False, index=False)
    return predict_data, output_data

#call the function predict_nb()
predict_data, output_data = predict_nb(predict_data)
output_data['output'].value_counts()



accuracy score:  0.8612546992481203
Variance score: 0.8612546992481203
Mean Absolute Error: 0.13874530075187969
Mean Squared Error: 0.13874530075187969
Root Mean Squared Error: 0.3724853027327114


TypeA    2385
TypeB     318
Name: output, dtype: int64